In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_02_01'
posner_id ='pos1a'
posner_id_b ='pos1b'

IMPORTED MODULES today: 2023-07-17 14:45:10.745022


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_02_01/m_02_01_pos1a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '1a' in file: m_02_01/02_1a_posner_task_2023-04-20_15h58.14.490/02_1a_posner_task_2023-04-20_15h58.14.490.csv
Found '1b' in file: m_02_01/02_1b_posner_task_2023-04-20_16h51.48.739/02_1b_posner_task_2023-04-20_16h51.48.739.csv
********** 1a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-04-20 15:58:49.870871+00:00 - 2023-04-20 16:13:24.090871+00:00
  [0] Keeping  (2023-04-20 15:58:49.870871+00:00 - 2023-04-20 15:58:49.870871+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-20 16:00:02.939871+00:00 - 2023-04-20 16:00:02.939871+00:00 -> 73.069 - 7

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [380] Keeping  (2023-04-20 16:10:44.967871+00:00 - 2023-04-20 16:10:44.967871+00:00 -> 715.097 - 715.097)
  [381] Keeping  (2023-04-20 16:10:48.087871+00:00 - 2023-04-20 16:10:48.087871+00:00 -> 718.217 - 718.217)
  [382] Keeping  (2023-04-20 16:10:48.187871+00:00 - 2023-04-20 16:10:48.187871+00:00 -> 718.317 - 718.317)
  [383] Keeping  (2023-04-20 16:10:49.967871+00:00 - 2023-04-20 16:10:49.967871+00:00 -> 720.097 - 720.097)
  [384] Keeping  (2023-04-20 16:10:51.467871+00:00 - 2023-04-20 16:10:51.467871+00:00 -> 721.597 - 721.597)
  [385] Keeping  (2023-04-20 16:10:54.747871+00:00 - 2023-04-20 16:10:54.747871+00:00 -> 724.877 - 724.877)
  [386] Keeping  (2023-04-20 16:10:54.847871+00:00 - 2023-04-20 16:10:54.847871+00:00 -> 724.977 - 724.977)
  [387] Keeping  (2023-04-20 16:10:56.467871+00:00 - 2023-04-20 16:10:56.467871+00:00 -> 726.597 - 726.597)
  [388] Keeping  (2023-04-20 16:10:57.967871+00:00 - 2023-04-20 16:10:57.967871+00:00 -> 728.097 - 728.097)
  [389] Keeping  (2023-04-20

In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
414 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 414 events and 701 original time points ...
5 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [6]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Fitting ICA took 595.0s.


Method,fastica
Fit,1000 iterations on raw data (680041 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [7]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=680041
    Range : 88023 ... 768063 =     88.023 ...   768.063 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items ac

In [8]:
print(ica.exclude)

[40, 42, 20, 39, 30, 22, 0, 1, 5, 13, 17, 19, 18, 15]


In [9]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 14 ICA components
    Projecting back using 63 PCA components
Using qt as 2D backend.
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


In [12]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1, overwrite=True)

m_02_01_pos1a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_02_01

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_02_01/m_02_01_pos1a.vhdr
Overwriting existing file.


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [13]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_02_01
├── m_02_01_pos1a.eeg
├── m_02_01_pos1a.vhdr
└── m_02_01_pos1a.vmrk

1 directory, 3 files


In [14]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [15]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_02_01/m_02_01_pos1b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '1a' in file: m_02_01/02_1a_posner_task_2023-04-20_15h58.14.490/02_1a_posner_task_2023-04-20_15h58.14.490.csv
Found '1b' in file: m_02_01/02_1b_posner_task_2023-04-20_16h51.48.739/02_1b_posner_task_2023-04-20_16h51.48.739.csv
********** 1b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-04-20 16:53:59.855341+00:00 - 2023-04-20 17:06:48.815341+00:00
  [0] Keeping  (2023-04-20 16:53:59.855341+00:00 - 2023-04-20 16:53:59.855341+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-20 16:54:00.528341+00:00 - 2023-04-20 16:54:00.528341+00:00 -> 0.673 - 0.673)
  [2] Keeping  (2023-04-20 16:54:02.028341+00:00 - 2023-04-20 1

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [389] Keeping  (2023-04-20 17:05:54.883341+00:00 - 2023-04-20 17:05:54.883341+00:00 -> 715.028 - 715.028)
  [390] Keeping  (2023-04-20 17:05:56.103341+00:00 - 2023-04-20 17:05:56.103341+00:00 -> 716.248 - 716.248)
  [391] Keeping  (2023-04-20 17:05:57.603341+00:00 - 2023-04-20 17:05:57.603341+00:00 -> 717.748 - 717.748)
  [392] Keeping  (2023-04-20 17:06:00.163341+00:00 - 2023-04-20 17:06:00.163341+00:00 -> 720.308 - 720.308)
  [393] Keeping  (2023-04-20 17:06:00.263341+00:00 - 2023-04-20 17:06:00.263341+00:00 -> 720.408 - 720.408)
  [394] Keeping  (2023-04-20 17:06:02.603341+00:00 - 2023-04-20 17:06:02.603341+00:00 -> 722.748 - 722.748)
  [395] Keeping  (2023-04-20 17:06:04.103341+00:00 - 2023-04-20 17:06:04.103341+00:00 -> 724.248 - 724.248)
  [396] Keeping  (2023-04-20 17:06:07.083341+00:00 - 2023-04-20 17:06:07.083341+00:00 -> 727.228 - 727.228)
  [397] Keeping  (2023-04-20 17:06:07.183341+00:00 - 2023-04-20 17:06:07.183341+00:00 -> 727.328 - 727.328)
  [398] Keeping  (2023-04-20

In [16]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
404 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 404 events and 701 original time points ...
3 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [17]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
os.system("say 'Hey you! The I C A is finished!'")
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-04-20 16:54:00.528341+00:00 - 2023-04-20 17:06:17.613341+00:00
  [0] Keeping  (2023-04-20 16:54:00.528341+00:00 - 2023-04-20 16:54:00.528341+00:00 -> 0.673 - 0.673)
  [1] Keeping  (2023-04-20 16:54:02.028341+00:00 - 2023-04-20 16:54:02.028341+00:00 -> 2.173 - 2.173)
  [2] Keeping  (2023-04-20 16:54:05.058341+00:00 - 2023-04-20 16:54:05.058341+00:00 -> 5.203 - 5.203)
  [3] Keeping  (2023-04-20 16:54:05.158341+00:00 - 2023-04-20 16:54:05.158341+00:00 -> 5.303 - 5.303)
  [4] Keeping  (2023-04-20 16:54:07.028341+00:00 - 2023-04-20 16:54:07.028341+00:00 -> 7.173 - 7.173)
  [5] Keeping  (2023-04-20 16:54:08.528341+00:00 - 2023-04-20 16:54:08.528341+00:00 -> 8.673 - 8.673)
  [6] Keeping  (2023-04-20 16:54:11.608341+00:00 - 2023-04-20 16:54:11.608341+00:00 -> 11.753 - 11.753)
  [7] Keeping  (2023-04-20 16:54:11.708341+00:00 - 2023-04-20 16:54:11.708341+0

Method,fastica
Fit,106 iterations on raw data (737085 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [18]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=737085
    Range : 673 ... 737757 =      0.673 ...   737.757 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
368 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
368 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
368 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
368 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
368 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
368 matching events found
No baseline correction applied
0 projection items ac

In [19]:
ica_b.exclude

[57, 44, 20, 21, 22, 23, 24, 26, 34, 36, 38, 1, 3, 13, 18, 0]

In [20]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 16 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [21]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_02_01/m_02_01_pos1b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [22]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_02_01
├── m_02_01_pos1a.eeg
├── m_02_01_pos1a.vhdr
├── m_02_01_pos1a.vmrk
├── m_02_01_pos1b.eeg
├── m_02_01_pos1b.vhdr
└── m_02_01_pos1b.vmrk

1 directory, 6 files


# ...

In [23]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b